In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from database.connection import sql_connection
from database.src import dataset_config

import lightgbm as lgb
import numpy as np
import pandas as pd
from modules.hyperparameters_opt.bayesian_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from skopt.space import Real, Integer
from modules.calibration_algorithm.interface import RunModel, RunOptimization

In [2]:
stdrings = sql_connection.set_standard_rings_data()
params=[[{'cluster_eta':[0.6, 0.8]}, {'cluster_et':[20000,30000]}],
        [{'cluster_eta':[1.2, 1.8]}, {'cluster_et':[100000, 250000]}]]

dataset=dataset_config.split_dataframe(stdrings, params)

group_conditions:  [{'cluster_eta': [0.6, 0.8]}, {'cluster_et': [20000, 30000]}]
condition_dict:  {'cluster_eta': [0.6, 0.8]}
col_name:  cluster_eta
intervals:  [0.6, 0.8]
intervals[0]:  0.6 intervals[1]:  0.8
TESTANDO
NEW COLUMNS
INFO: Aplicando comparação com módulo para a coluna 'cluster_eta'.
condition_dict:  {'cluster_et': [20000, 30000]}
col_name:  cluster_et
intervals:  [20000, 30000]
intervals[0]:  20000 intervals[1]:  30000
TESTANDO
NEW COLUMNS
group_conditions:  [{'cluster_eta': [1.2, 1.8]}, {'cluster_et': [100000, 250000]}]
condition_dict:  {'cluster_eta': [1.2, 1.8]}
col_name:  cluster_eta
intervals:  [1.2, 1.8]
intervals[0]:  1.2 intervals[1]:  1.8
TESTANDO
NEW COLUMNS
INFO: Aplicando comparação com módulo para a coluna 'cluster_eta'.
condition_dict:  {'cluster_et': [100000, 250000]}
col_name:  cluster_et
intervals:  [100000, 250000]
intervals[0]:  100000 intervals[1]:  250000
TESTANDO
NEW COLUMNS


In [3]:
print(type(dataset))
print(len(dataset))
print(len(dataset[0]))
print(len(dataset[1]))
print(len(dataset[2]))
print(type(dataset[0]))
print((dataset[0].columns))

<class 'list'>
3
2056
59840
468114
<class 'pandas.core.frame.DataFrame'>
Index(['StdRings_0', 'StdRings_1', 'StdRings_2', 'StdRings_3', 'StdRings_4',
       'StdRings_5', 'StdRings_6', 'StdRings_7', 'StdRings_8', 'StdRings_9',
       ...
       'cluster_phi', 'cluster_et', 'cluster_e237', 'cluster_e277', 'mc_et',
       'delta_eta_calib', 'delta_phi_calib', 'hotCellEta', 'hotCellPhi',
       'alpha'],
      dtype='object', length=111)


In [4]:
data = dataset_config.prepare_and_split_data(input_type='std_rings',stdrings_df=dataset)
print(type(data))
print(len(data))
print(type(data[0]))

<class 'list'>
3
<class 'tuple'>


In [5]:
space_lgbm = [
    Integer(20, 100, name='num_leaves'),
    Integer(3, 15, name='max_depth'),
    Real(0.01, 0.2, name='learning_rate', prior='log-uniform')
]

fixed_params_lgbm = {
    'objective': 'mae', 
    'metric': 'mae', 
    'num_iterations ': 1000,
    'random_state': 42, 
    'n_jobs': -1, 
    'verbose': -1
}

calibration_kwargs = {'callbacks':[lgb.early_stopping(stopping_rounds=10, verbose=False)]}
optimization_kwargs = {'n_initial_points': 15,
                        'n_calls': 30,
                        'initial_point_generator': 'random',
                        'random_state': 42,

                       
                       }

# Simplesmente passe 'model_type="lightgbm"'
optimizer_lgbm = RunOptimization().run_multiple_optimizations(opt_class='gp_minimize',
    model_class="lgbm",
    datasets=data,
    space=space_lgbm, 
    fixed_params=fixed_params_lgbm,
    metric='mae',
    calibration_kwargs=calibration_kwargs,
    optimization_kwargs= optimization_kwargs

)

print(f"\nMelhor MAE para LightGBM: {optimizer_lgbm[0].fun:.4f}")
print(f"Melhores parâmetros: {optimizer_lgbm[0].x}")


[INFO] Running optimization 4
<class 'tuple'>
<class 'pandas.core.frame.DataFrame'>


ValueError: DataFrame for label cannot have multiple columns